# Remaining Useful Life (RUL) Estimation from Strain DIC Measurement

This notebook demonstrates the workflow for estimating the remaining useful life (RUL) of a tungsten component using Digital Image Correlation (DIC) strain data.

**Author:** Jayron Sandhu

## Setup and Imports

First, let's import the required modules and libraries.

In [ ]:
# Import necessary modules
import numpy as np
import matplotlib.pyplot as plt
import os
import rainflow
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Import modules from our project
from data_loader import load_all_data, print_statistical_summary
from plotter import (plot_initial_strain_analysis, plot_stress_analysis, 
                    plot_strain_analysis, plot_fatigue_analysis_signals,
                    plot_rul_estimation)
from fatigue_analysis import (calculate_stress, calculate_principal_strains,
                             identify_cycles, analyze_fatigue,
                             estimate_fatigue_life)

# Set matplotlib parameters for better visualization
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 12

## 1. Load and Process Strain Data

Digital Image Correlation (DIC) provides full-field displacement and strain measurements by comparing images of a specimen at different loading stages. Here, we're loading strain data from the DIC measurements of a tungsten component under thermal cycling.

In [ ]:
# Load strain data
print("Loading strain data...")
data = load_all_data()

# Extract data components
if isinstance(data, dict):
    # Use dictionary format
    ThermalStrain = data['thermal_strain']
    DICExx = data['strain_exx']
    DICEyy = data['strain_eyy']
    time_points = data['time_points']
    high_strain_points = data['high_strain_points']
    mean_strain = data['mean_strain']
    max_strain = data['max_strain'] 
    std_strain = data['std_strain']
else:
    # Use tuple format for backward compatibility
    _, DICExx, _, DICEyy, ThermalStrain, time_points, mean_strain, std_strain, max_strain, high_strain_points = data

# Print statistical summary
print_statistical_summary(ThermalStrain, high_strain_points)

## 2. Visualize Initial Strain Data

Let's visualize the strain data to understand the distribution and patterns. This will help identify high-strain regions and potential failure points.

In [ ]:
# Plot initial strain patterns
plot_initial_strain_analysis(DICExx, DICEyy, ThermalStrain, time_points, high_strain_points)

print("\nInitial strain analysis complete.\n")
print(f"Maximum strain magnitude: {max_strain:.2e}")
print(f"Mean strain magnitude: {mean_strain:.2e}")
print(f"Strain standard deviation: {std_strain:.2e}")

## 3. Calculate and Visualize Stress Components

Using Hooke's law, we can convert strain measurements to stress. For tungsten, which has isotropic mechanical properties, we use its Young's modulus and Poisson's ratio for this conversion.

In [ ]:
# Define material properties for tungsten
E = 400e3  # Young's modulus in MPa (400 GPa)
nu = 0.28  # Poisson's ratio
yield_strength = 1000.0  # Approximate yield strength in MPa

# Calculate stress components
sigma_xx, sigma_yy, von_mises_stress, safety_factor = calculate_stress(DICExx, DICEyy, E, nu, yield_strength)

# Plot stress components and safety factor
plot_stress_analysis(sigma_xx, sigma_yy, von_mises_stress, safety_factor, high_strain_points)

# Print stress analysis summary
print("\nStress analysis complete.\n")
print(f"Maximum σxx: {np.nanmax(sigma_xx):.2f} MPa")
print(f"Maximum σyy: {np.nanmax(sigma_yy):.2f} MPa")
print(f"Maximum von Mises stress: {np.nanmax(von_mises_stress):.2f} MPa")
print(f"Yield strength of tungsten: ~{yield_strength:.1f} MPa")
print(f"Safety factor: {np.nanmin(safety_factor):.2f}")

## 4. Calculate and Visualize Principal Strains

Principal strains represent the maximum and minimum strains at each point, regardless of direction. They are critical for fatigue analysis.

In [ ]:
## 6. Estimate and Visualize Remaining Useful Life (RUL)

Using the identified strain cycles, we can estimate the remaining useful life (RUL) of the component, based on the Manson-Coffin relationship and Miner's rule of cumulative damage. We also calculate the elapsed time based on cycle duration to provide context for maintenance scheduling.e1, e2, max_shear, theta = calculate_principal_strains(DICExx, DICEyy)

# Plot principal strain components
plot_strain_analysis(e1, e2, max_shear, theta, high_strain_points)

# Print principal strain analysis summary
print("\nPrincipal strain analysis complete.\n")
print(f"Maximum principal strain (e1): {np.nanmax(e1):.2e}")
print(f"Minimum principal strain (e2): {np.nanmin(e2):.2e}")
print(f"Maximum shear strain: {np.nanmax(max_shear):.2e}")

# Estimate RUL for principal strain
print("Estimating remaining useful life...")
x_rul_e1, y_rul_e1, initial_rul_e1, final_rul_e1, life_used_percentage_e1 = estimate_fatigue_life(
    fatigue_results_e1, cycle_multiplier=10)

# Estimate RUL for max shear strain
x_rul_max_shear, y_rul_max_shear, initial_rul_max_shear, final_rul_max_shear, life_used_percentage_max_shear = estimate_fatigue_life(
    fatigue_results_max_shear, cycle_multiplier=10, force_shear=True)

# Time duration per cycle (from data_loader.py)
time_per_cycle = 0.2  # seconds

# Plot RUL estimations with time per cycle parameter
plot_rul_estimation(x_rul_e1, y_rul_e1, x_rul_max_shear, y_rul_max_shear, 
                   max_principal_loc, max_shear_loc, time_per_cycle=time_per_cycle)

# Print RUL estimation summary
print("\nRUL estimation complete.\n")
print(f"Principal Strain - Initial RUL: {initial_rul_e1:.1f} cycles, Final RUL: {final_rul_e1:.1f} cycles")
print(f"Principal Strain - Life used: {life_used_percentage_e1:.2f}%")
print(f"Max Shear Strain - Initial RUL: {initial_rul_max_shear:.1f} cycles, Final RUL: {final_rul_max_shear:.1f} cycles")
print(f"Max Shear Strain - Life used: {life_used_percentage_max_shear:.2f}%")
Rainflow cycle counting is a technique used in fatigue analysis to count strain cycles in irregular strain-time histories. This is crucial for accurate fatigue life prediction.

In [ ]:
# Identify cycles in principal strain signals
print("Performing rainflow cycle counting...")
cycles_e1 = identify_cycles(e1, high_strain_points)
cycles_max_shear = identify_cycles(max_shear, high_strain_points, is_shear_strain=True)

# Extract and analyze fatigue results
fatigue_results_e1 = analyze_fatigue(e1, high_strain_points, cycles_e1, 'Principal Strain')
fatigue_results_max_shear = analyze_fatigue(max_shear, high_strain_points, cycles_max_shear, 'Max Shear Strain')

# Plot fatigue analysis signals
plot_fatigue_analysis_signals(fatigue_results_e1, fatigue_results_max_shear)

# Print fatigue analysis summary
print("\nFatigue analysis complete.\n")
print(f"Principal strain cycles identified: {len(cycles_e1)}")
print(f"Shear strain cycles identified: {len(cycles_max_shear)}")

## 6. Estimate and Visualize Remaining Useful Life (RUL)

Using the identified strain cycles, we can estimate the remaining useful life (RUL) of the component, based on the Manson-Coffin relationship and Miner's rule of cumulative damage.

In [ ]:
# Estimate RUL for principal strain
print("Estimating remaining useful life...")
x_rul_e1, y_rul_e1, initial_rul_e1, final_rul_e1, life_used_percentage_e1 = estimate_fatigue_life(
    fatigue_results_e1, cycle_multiplier=10)

# Estimate RUL for max shear strain
x_rul_max_shear, y_rul_max_shear, initial_rul_max_shear, final_rul_max_shear, life_used_percentage_max_shear = estimate_fatigue_life(
    fatigue_results_max_shear, cycle_multiplier=10, force_shear=True)

# Plot RUL estimations
plot_rul_estimation(x_rul_e1, y_rul_e1, x_rul_max_shear, y_rul_max_shear)

# Print RUL estimation summary
print("\nRUL estimation complete.\n")
print(f"Principal Strain - Initial RUL: {initial_rul_e1:.1f} cycles, Final RUL: {final_rul_e1:.1f} cycles")
print(f"Principal Strain - Life used: {life_used_percentage_e1:.2f}%")
print(f"Max Shear Strain - Initial RUL: {initial_rul_max_shear:.1f} cycles, Final RUL: {final_rul_max_shear:.1f} cycles")
print(f"Max Shear Strain - Life used: {life_used_percentage_max_shear:.2f}%")

## 7. Conclusion and Recommendations

Based on the fatigue analysis and RUL estimation, we can draw the following conclusions:

1. The tungsten component is undergoing significant strain cycling, particularly in regions of high strain concentration.
2. The principal strain analysis indicates a higher rate of life consumption compared to the shear strain analysis.
3. Based on our RUL calculations, the component has used approximately:
   - For principal strain: The displayed percentage of its fatigue life
   - For shear strain: The displayed percentage of its fatigue life

**Recommendations:**

1. If the life used percentage exceeds 50% for either strain type, consider scheduling maintenance or replacement.
2. Continue monitoring the component for changes in strain patterns that might accelerate fatigue damage.
3. Consider design modifications to reduce strain concentrations in critical areas.
4. For more accurate RUL estimation, collect additional cycles of data and refine material parameters.

This analysis demonstrates the power of combining DIC strain measurements with fatigue analysis techniques to predict component life and inform maintenance decisions.